# Building a Random Forest:


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv('LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,24700.0,24700.0,24700.0,36 months,11.99%,820.28,C,C1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,35000.0,35000.0,35000.0,60 months,14.85%,829.90,C,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3600.0,3600.0,3600.0,36 months,13.99%,123.03,C,C4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,10400.0,10400.0,10400.0,60 months,22.45%,289.91,F,F1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,20000.0,20000.0,20000.0,60 months,10.78%,432.66,B,B4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


## Blind Approach
Try putting everything into a Random Forest without much preprocessing. SKLearn needs independent variables to be numeric, use get_dummies to generate a dummy variable for every categorical column



In [5]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status',1)
Y = y2015['loan_status']
#X = pd.get_dummies(X) #this kills the kernel

#cross_val_score(rfc,X,Y,cv=5)


In [6]:
y2015.shape

(421097, 144)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [7]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
2
term
2
int_rate
111
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
2
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
53
next_pymnt_d
5
last_credit_pull_d
54
application_type
2
verification_status_joint
1
hardship_flag
2
hardship_type
1
hardship_reason
9
hardship_status
3
hardship_start_date
28
hardship_end_date
29
payment_plan_start_date
29
hardship_loan_status
4
debt_settlement_flag
2
debt_settlement_flag_date
46
settlement_status
3
settlement_date
49


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [8]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which should have all been integers? Let's look at the end of the file.

In [9]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
421092,NaN,NaN,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421093,NaN,NaN,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421094,NaN,NaN,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [11]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,settlement_date_Nov-2017,settlement_date_Nov-2018,settlement_date_Oct-2015,settlement_date_Oct-2016,settlement_date_Oct-2017,settlement_date_Oct-2018,settlement_date_Sep-2015,settlement_date_Sep-2016,settlement_date_Sep-2017,settlement_date_Sep-2018
0,NaN,NaN,24700.0,24700.0,24700.0,11.99,820.28,65000.0,16.06,1.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,35000.0,35000.0,35000.0,14.85,829.90,110000.0,17.06,0.0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,3600.0,3600.0,3600.0,13.99,123.03,55000.0,5.91,0.0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,10400.0,10400.0,10400.0,22.45,289.91,104433.0,25.37,1.0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,20000.0,20000.0,20000.0,10.78,432.66,63000.0,10.78,0.0,...,0,0,0,0,0,0,0,0,0,0
5,NaN,NaN,11950.0,11950.0,11950.0,13.44,405.18,34000.0,10.20,0.0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,NaN,16000.0,16000.0,16000.0,12.88,363.07,70000.0,26.40,0.0,...,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,20000.0,20000.0,20000.0,9.17,637.58,180000.0,14.67,0.0,...,0,0,0,0,0,0,0,0,0,0
8,NaN,NaN,1400.0,1400.0,1400.0,12.88,47.10,64000.0,34.95,0.0,...,0,0,0,0,0,0,0,0,0,0
9,NaN,NaN,18000.0,18000.0,18000.0,19.48,471.70,150000.0,9.39,0.0,...,0,0,0,0,0,0,0,0,0,0


It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [78]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklea

array([0.98945669, 0.99076251, 0.99142742, 0.99306594, 0.99344574,
       0.99327935, 0.99337434, 0.99370666, 0.99415774, 0.9926853 ])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [36]:

#drop variables related to payments/principle
X_2 = X.drop(['out_prncp',
              'out_prncp_inv',
              'total_pymnt',
              'total_pymnt_inv',
              'last_pymnt_amnt',
              'funded_amnt',
              'funded_amnt_inv',
              'installment',
              'loan_amnt',
              'tot_coll_amt',
              'tot_cur_bal'],1)
# X_2 = pd.get_dummies(X_2)
X_2 = X_2.dropna(axis=1)
cross_val_score(rfc,X_2,Y,cv=10)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklea

array([0.97352299, 0.97269122, 0.97053027, 0.97575455, 0.97409166,
       0.97774822, 0.97727327, 0.97591907, 0.97660721, 0.97729594])

In [55]:
#drop variables that seem unlikely to predict loan status
#drop issue date
X_3 = X_2.drop(['issue_d_Apr-2015',
                'issue_d_Aug-2015',
                'issue_d_Dec-2015',
                'issue_d_Feb-2015',
                'issue_d_Jan-2015',
                'issue_d_Jul-2015',
                'issue_d_Jun-2015',
                'issue_d_Mar-2015',
                'issue_d_May-2015',
                'issue_d_Nov-2015',
                'issue_d_Oct-2015',
                'issue_d_Sep-2015'],1)
X_3 = pd.get_dummies(X_3)
X_3 = X_3.dropna(axis=1)
cross_val_score(rfc,X_3,Y,cv=10)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklea

array([0.97235942, 0.97200256, 0.97494716, 0.97508964, 0.97696509,
       0.97772448, 0.97551592, 0.97667902, 0.97667846, 0.97786591])

In [76]:
#drop payment due dates, credit pull dates

# first, creating a column of all the credit pulls and payment due dates, which are 
# formatted as "last_payment" or "last_credit_pull"
cols = X_3.columns.to_list()
payment_cols = [s for s in cols if "last" in s]
payment_cols = payment_cols[1:]
#payment_cols




In [75]:
X_4 = X_3.drop(['last_pymnt_d_Apr-2015',
 'last_pymnt_d_Apr-2016',
 'last_pymnt_d_Apr-2017',
 'last_pymnt_d_Apr-2018',
 'last_pymnt_d_Apr-2019',
 'last_pymnt_d_Aug-2015',
 'last_pymnt_d_Aug-2016',
 'last_pymnt_d_Aug-2017',
 'last_pymnt_d_Aug-2018',
 'last_pymnt_d_Dec-2015',
 'last_pymnt_d_Dec-2016',
 'last_pymnt_d_Dec-2017',
 'last_pymnt_d_Dec-2018',
 'last_pymnt_d_Feb-2015',
 'last_pymnt_d_Feb-2016',
 'last_pymnt_d_Feb-2017',
 'last_pymnt_d_Feb-2018',
 'last_pymnt_d_Feb-2019',
 'last_pymnt_d_Jan-2015',
 'last_pymnt_d_Jan-2016',
 'last_pymnt_d_Jan-2017',
 'last_pymnt_d_Jan-2018',
 'last_pymnt_d_Jan-2019',
 'last_pymnt_d_Jul-2015',
 'last_pymnt_d_Jul-2016',
 'last_pymnt_d_Jul-2017',
 'last_pymnt_d_Jul-2018',
 'last_pymnt_d_Jun-2015',
 'last_pymnt_d_Jun-2016',
 'last_pymnt_d_Jun-2017',
 'last_pymnt_d_Jun-2018',
 'last_pymnt_d_Mar-2015',
 'last_pymnt_d_Mar-2016',
 'last_pymnt_d_Mar-2017',
 'last_pymnt_d_Mar-2018',
 'last_pymnt_d_Mar-2019',
 'last_pymnt_d_May-2015',
 'last_pymnt_d_May-2016',
 'last_pymnt_d_May-2017',
 'last_pymnt_d_May-2018',
 'last_pymnt_d_May-2019',
 'last_pymnt_d_Nov-2015',
 'last_pymnt_d_Nov-2016',
 'last_pymnt_d_Nov-2017',
 'last_pymnt_d_Nov-2018',
 'last_pymnt_d_Oct-2015',
 'last_pymnt_d_Oct-2016',
 'last_pymnt_d_Oct-2017',
 'last_pymnt_d_Oct-2018',
 'last_pymnt_d_Sep-2015',
 'last_pymnt_d_Sep-2016',
 'last_pymnt_d_Sep-2017',
 'last_pymnt_d_Sep-2018',
 'last_credit_pull_d_Apr-2015',
 'last_credit_pull_d_Apr-2016',
 'last_credit_pull_d_Apr-2017',
 'last_credit_pull_d_Apr-2018',
 'last_credit_pull_d_Apr-2019',
 'last_credit_pull_d_Aug-2015',
 'last_credit_pull_d_Aug-2016',
 'last_credit_pull_d_Aug-2017',
 'last_credit_pull_d_Aug-2018',
 'last_credit_pull_d_Dec-2014',
 'last_credit_pull_d_Dec-2015',
 'last_credit_pull_d_Dec-2016',
 'last_credit_pull_d_Dec-2017',
 'last_credit_pull_d_Dec-2018',
 'last_credit_pull_d_Feb-2015',
 'last_credit_pull_d_Feb-2016',
 'last_credit_pull_d_Feb-2017',
 'last_credit_pull_d_Feb-2018',
 'last_credit_pull_d_Feb-2019',
 'last_credit_pull_d_Jan-2015',
 'last_credit_pull_d_Jan-2016',
 'last_credit_pull_d_Jan-2017',
 'last_credit_pull_d_Jan-2018',
 'last_credit_pull_d_Jan-2019',
 'last_credit_pull_d_Jul-2015',
 'last_credit_pull_d_Jul-2016',
 'last_credit_pull_d_Jul-2017',
 'last_credit_pull_d_Jul-2018',
 'last_credit_pull_d_Jun-2015',
 'last_credit_pull_d_Jun-2016',
 'last_credit_pull_d_Jun-2017',
 'last_credit_pull_d_Jun-2018',
 'last_credit_pull_d_Mar-2015',
 'last_credit_pull_d_Mar-2016',
 'last_credit_pull_d_Mar-2017',
 'last_credit_pull_d_Mar-2018',
 'last_credit_pull_d_Mar-2019',
 'last_credit_pull_d_May-2015',
 'last_credit_pull_d_May-2016',
 'last_credit_pull_d_May-2017',
 'last_credit_pull_d_May-2018',
 'last_credit_pull_d_May-2019',
 'last_credit_pull_d_Nov-2015',
 'last_credit_pull_d_Nov-2016',
 'last_credit_pull_d_Nov-2017',
 'last_credit_pull_d_Nov-2018',
 'last_credit_pull_d_Oct-2015',
 'last_credit_pull_d_Oct-2016',
 'last_credit_pull_d_Oct-2017',
 'last_credit_pull_d_Oct-2018',
 'last_credit_pull_d_Sep-2015',
 'last_credit_pull_d_Sep-2016',
 'last_credit_pull_d_Sep-2017',
 'last_credit_pull_d_Sep-2018'], 1)
X_4 = pd.get_dummies(X_4)
X_4 = X_4.dropna(axis=1)
cross_val_score(rfc,X_4,Y,cv=10)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklea

array([0.96810885, 0.9680606 , 0.96950915, 0.97036404, 0.96957967,
       0.97190624, 0.97159752, 0.97086064, 0.97019498, 0.97211865])

In [77]:
#customer entered title descriptions
X_5 = X_4.drop(['title_Car financing',
                'title_Credit Card/Auto Repair',
                'title_Credit card refinancing',
                'title_Debt consolidation',
                'title_DebtC',
                'title_Green loan',
                'title_Home buying',
                'title_Home improvement',
                'title_Learning and training',
                'title_Major purchase',
                'title_Medical expenses',
                'title_Moving and relocation',
                'title_New Baby and New House (CC Consolidate)',
                'title_Other',
                'title_Pay off Lowes Card',
                'title_Paying off higher interest cards & auto',
                'title_Prescription Drug and Medical Costs',
                'title_SAVE',
                'title_Simple Loan Until Contract Is Completed',
                'title_Student Loan',
                'title_Trying to come back to reality!',
                'title_Vacation',
                'title_considerate',
                'title_new day',
                'title_new kitchen for momma!',
                'title_odymeds',
                'title_Business'],1)
X_5 = pd.get_dummies(X_5)
X_5 = X_5.dropna(axis=1)
cross_val_score(rfc,X_5,Y,cv=10)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklea

array([0.96732523, 0.97055401, 0.9698891 , 0.96777564, 0.97219188,
       0.97333112, 0.97138379, 0.96746461, 0.97069371, 0.96979125])

In [82]:
#drop misc variables
X_6 = X_5.drop(['policy_code',
                'verification_status_Not Verified',
                'verification_status_Source Verified',
                'verification_status_Verified',
                'delinq_2yrs',
                'inq_last_6mths',
                'open_acc',
                'pub_rec',
                'revol_bal',
                'total_acc',
                'total_rec_prncp',
                'total_rec_int',
                'total_rec_late_fee',
                'recoveries',
                'collection_recovery_fee',
                'collections_12_mths_ex_med',
                'next_pymnt_d_Dec-2018',
                'next_pymnt_d_Jul-2019',
                'next_pymnt_d_Jun-2019',
                'next_pymnt_d_May-2019',
                'next_pymnt_d_Sep-2018'],1)
X_6 = pd.get_dummies(X_6)
X_6 = X_6.dropna(axis=1)
cross_val_score(rfc,X_6,Y,cv=10)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklea

array([0.74081022, 0.737622  , 0.73524732, 0.73156657, 0.7320114 ,
       0.72566435, 0.72452445, 0.72076565, 0.72401263, 0.71593797])

This droppped the score a little too low
